# 0.0) IMPORTS

In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px

import inflection

## 0.1) Helper Variables

In [2]:
PATH = "dataset/zomato.csv"

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}


COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

## 0.2) Helper Functions

In [4]:
def RenameColumns(df):
    cols = []
    for i in df1.columns:
        cols.append(i.replace(' ', '_').lower().lstrip().rstrip())
    df.columns= cols

    return df

In [5]:
def DataframeDimensions(df):
    print(f"Número de Linhas: {df.shape[0]}")
    print(f"Número de Colunas: {df.shape[1]}")
    
    return None

In [6]:
def GetNumericalAttributes(df):
    return df.select_dtypes(include=['int64', 'float64'])

In [7]:
def Statistics(df):
    # Central Tendency Metrics
    mean = pd.DataFrame(df.apply(np.mean)).T
    median = pd.DataFrame(df.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(df.apply(min)).T
    max_ = pd.DataFrame(df.apply(max)).T
    range_ = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(df.apply(np.std)).T
    skew = pd.DataFrame(df.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return m

In [8]:
def ColorName(color_code):
    return COLORS[color_code]

In [9]:
def CountryName(country_id):
    return COUNTRIES[country_id]

In [10]:
def CreatePriceTye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [11]:
def ColumnsOrder(df):
    df = df.copy()

    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]

    return df.loc[:, new_cols_order]

In [12]:
def PrepareData(path):
    df = pd.read_csv(path)
    
    df = RenameColumns(df)

    df = df.dropna()
    
    df = df.drop_duplicates()
    
    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: CreatePriceTye(x))

    df["country"] = df.loc[:, "country_code"].apply(lambda x: CountryName(x))

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: ColorName(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])
 
    df = ColumnsOrder(df)
    
    df.to_csv("dataset/processed/data.csv", index=False)

    return df

## 0.3) Load Data

In [13]:
df_raw = pd.read_csv(PATH)
df_raw.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


# 1.0) DATA DESCRIPTION

In [14]:
df1 = df_raw.copy()
df1.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


## 1.1) Columns Description

| Column | Description |
| :----- | :---------- |
| Restaurant ID | ID do restaurante | 
| Restaurant Name | Nome do Restaurante |
| Country Code | Código do País |
| City | Nome da Cidade onde o restaurante está |
| Address | Endereço do restaurante |
| Locality | Localização e pontos de referência do restaurante |
| Locality Verbose | Localização e pontos de referência do restaurante (Mais informações) |
| Longitude | Ponto geográfico de Longitude do Restaurante |
| Latitude | Ponto geográfico de Latitude do Restaurante |
| Cuisines | Tipos de Culinária servidos no restaurante |
| Average Cost for two | Preço Médio de um prato para duas pessoas no restaurante |
| Currency | Moeda do país |
| Has Table booking | Se o restaurante possui serviços de reserva; 1 - Sim; 0 - Não |
| Has Online delivery | Se o restaurante possui serviços de pedido on-line; 1 - Sim; 0 - Não |
| Is delivering now | Se o restaurante faz entregas; 1 - Sim; 0 - Não |
| Switch to order menu | Se o restaurante alternou para menu de pedidos; 1 - Sim; 0 - Não |
| Price range | Variação de preços do restaurante; 1 a 4 - Quanto maior o valor, mais caro serão os pratos |
| Aggregate rating | Nota média do restaurante |
| Rating color | Código Hexadecimal da cor do restaurante com base em sua nota média |
| Rating text | Categoria em que o restaurante está com base em sua nota média |
| Votes | Quantidade de avaliações que o restaurante já recebeu |

## 1.2) Rename Columns

In [15]:
df1 = RenameColumns(df1)
df1.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


## 1.3) Data Dimensions

In [16]:
DataframeDimensions(df1)

Número de Linhas: 7527
Número de Colunas: 21


## 1.4) Data Types

In [17]:
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
dtype: object

## 1.5) Check NaN and Duplicated

In [18]:
df1.isna().sum()

restaurant_id            0
restaurant_name          0
country_code             0
city                     0
address                  0
locality                 0
locality_verbose         0
longitude                0
latitude                 0
cuisines                15
average_cost_for_two     0
currency                 0
has_table_booking        0
has_online_delivery      0
is_delivering_now        0
switch_to_order_menu     0
price_range              0
aggregate_rating         0
rating_color             0
rating_text              0
votes                    0
dtype: int64

In [19]:
df1.loc[:, 'cuisines'].nunique()

2832

In [20]:
df1.duplicated().sum()

585

## 1.6) Treat NaN an Duplicated

In [21]:
df1 = df1.dropna()
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

In [22]:
df1 = df1.drop_duplicates()

## 1.7) Change Types

Não há a necessidade de fazer a troca de tipos dos dados.

## 1.8) Descriptive Statistics

In [23]:
num_data = GetNumericalAttributes(df1)

### 1.8.1. Numerical Data

In [24]:
Statistics(num_data)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.074958e+07,7.900139e+06,7.037873e+06,-0.138598,-1.714864
1,country_code,1.000000,2.160000e+02,2.150000e+02,9.583966e+01,3.000000e+01,9.928432e+01,0.208034,-1.871897
2,longitude,-122.700455,1.753106e+02,2.980110e+02,3.145010e+01,7.375779e+01,7.902858e+01,-0.613523,-0.733448
3,latitude,-41.330428,5.597698e+01,9.730741e+01,1.941570e+01,2.526666e+01,2.351556e+01,-1.073323,0.542466
4,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.520077e+03,2.500000e+02,3.025129e+05,81.424161,6725.361462
5,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.061481e-02,0.000000e+00,2.386224e-01,3.683479,11.571359
6,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.504113e-01,0.000000e+00,4.770988e-01,0.627212,-1.607069
7,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.749170e-01,0.000000e+00,3.798961e-01,1.711802,0.930533
8,switch_to_order_menu,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
9,price_range,1.000000,4.000000e+00,3.000000e+00,2.633280e+00,3.000000e+00,9.656509e-01,-0.056585,-0.998100


# 2) Respostas as perguntas de negócio

In [25]:
df2 = PrepareData(PATH)
df2.head()

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,price_type,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,expensive,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,gourmet,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,expensive,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,expensive,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
5,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,expensive,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930


## Geral

### 1) Quantos restaurantes únicos estão registrados?

In [26]:
print("Estão registrados {} restaurantes únicos.".format(df2['restaurant_id'].nunique()))

Estão registrados 6929 restaurantes únicos.


### 2) Quantos paises únicos estão registrados?

In [27]:
print("Estão registrados {} países únicos.".format(df2['country'].nunique()))


Estão registrados 15 países únicos.


### 3) Quantas cidades únicas estão registradas?

In [28]:
print("Estão registradas {} cidades únicas.".format(df2['city'].nunique()))

Estão registradas 125 cidades únicas.


### 4) Qual o total de avaliações feitas?

In [29]:
print("O total de avaliações feitas são {:,}.".format(df2['votes'].sum()))

O total de avaliações feitas são 4,194,533.


### 5) Qual o total de tipos de culinária registrados?

In [30]:
print("O total de tipos de culinárias registrados são {}.".format(df2['cuisines'].nunique()))

O total de tipos de culinárias registrados são 165.


## País

### 1) Qual o nome do país que possui mais cidades registradas?

In [31]:
df_aux = (df2.loc[:, ['city', 'country'] ].groupby('country')
                                          .nunique()
                                          .sort_values('city', ascending=False)
                                          .reset_index())

print("O país com mais cidades registradas é {}.".format(df_aux.iloc[0,0]))


O país com mais cidades registradas é India.


### 2) Qual o nome do país que possui mais restaurantes registrados?

In [32]:
df_aux = (df2.loc[:, ['restaurant_id', 'country'] ].groupby('country')
                                                   .count()
                                                   .sort_values('restaurant_id', ascending=False)
                                                   .reset_index())
print("O país com mais restaurantes registrados é {}.".format(df_aux.iloc[0,0]))

O país com mais restaurantes registrados é India.


### 3) Qual o nome do país que possui mais restaurantes com os nível de preço igual a 4 registrados?

In [33]:
(df2.loc[ df2['aggregate_rating'] == 4, ['restaurant_id', 'country'] ]
                                         .groupby('country')
                                         .count()
                                         .sort_values('restaurant_id', ascending=False)
                                         .reset_index())
print("O país com mais restaurantes de nível de preço igual a 4 registrados é {}.".format(df_aux.iloc[0,0]))

O país com mais restaurantes de nível de preço igual a 4 registrados é India.


### 4) Qual o nome da país que possui a maior quantidade de tipos de culinária distintos?

In [34]:
df_aux = (df2.loc[:, ['cuisines', 'country'] ].groupby('country')
                                              .nunique()
                                              .sort_values('cuisines', ascending=False)
                                              .reset_index())
print("O país com a maior quantidade de tipos de culinária distintos é {}.".format(df_aux.iloc[0,0]))

O país com a maior quantidade de tipos de culinária distintos é India.


### 5) Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [35]:
df_aux = (df2.loc[:, ['votes', 'country'] ].groupby('country')
                                           .sum()
                                           .sort_values('votes', ascending=False)
                                           .reset_index())
print("O país que possui a maior quantidade de avaliações feitas é {}.".format(df_aux.iloc[0,0]))

O país que possui a maior quantidade de avaliações feitas é India.


### 6) Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [36]:
df_aux = (df2.loc[df2['is_delivering_now'] == 1, ['restaurant_id', 'country'] ].groupby('country')
                                                     .count()
                                                     .sort_values('restaurant_id', ascending=False)
                                                     .reset_index())
print("O país que possui a maior quantidade de restaurantes que fazem entrega é {}.".format(df_aux.iloc[0,0]))

O país que possui a maior quantidade de restaurantes que fazem entrega é India.


### 7) Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [37]:
df_aux = (df2.loc[df2['has_table_booking'] == 1, ['restaurant_id', 'country'] ].groupby('country')
                                                     .count()
                                                     .sort_values('restaurant_id', ascending=False)
                                                     .reset_index())
print("O país que possui a maior quantidade de restaurantes que que aceitam reservas é {}.".format(df_aux.iloc[0,0]))

O país que possui a maior quantidade de restaurantes que que aceitam reservas é India.


### 8) Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [38]:
df_aux = (df2.loc[:, ['votes', 'country'] ].groupby('country')
                                           .mean()
                                           .sort_values('votes', ascending=False)
                                           .reset_index())
print("O país que possui, na média, a maior quantidade de avaliações registradas é {}.".format(df_aux.iloc[0,0]))

O país que possui, na média, a maior quantidade de avaliações registradas é Indonesia.


### 9) Qual o nome do país que possui, na média, a maior nota média registrada?

In [39]:
df_aux = (df2.loc[:, ['aggregate_rating', 'country'] ].groupby('country')
                                                      .mean()
                                                      .sort_values('aggregate_rating', ascending=False)
                                                      .reset_index())
print("O país que possui, na média, a maior nota média registrada é {}.".format(df_aux.iloc[0,0]))

O país que possui, na média, a maior nota média registrada é Indonesia.


### 10) Qual o nome do país que possui, na média, a menor nota média registrada?

In [40]:
df_aux = (df2.loc[:, ['aggregate_rating', 'country'] ].groupby('country')
                                                      .mean()
                                                      .sort_values('aggregate_rating', ascending=True)
                                                      .reset_index())
print("O país que possui, na média, a menor nota média registrada é {}.".format(df_aux.iloc[0,0]))

O país que possui, na média, a menor nota média registrada é Brazil.


### 11) Qual a média de preço de um prato para dois por país?

In [41]:
df_aux = (df2.loc[:, ['currency','average_cost_for_two', 'country'] ].groupby('country')
                                                          .agg({'average_cost_for_two':'mean',
                                                                'currency':'min'})
                                                          .sort_values('average_cost_for_two', ascending=False)
                                                          .reset_index())
df_aux = df_aux.loc[:,['country','currency','average_cost_for_two']]
print("A média de preço de um prato para dois por país é:")
display(df_aux)

A média de preço de um prato para dois por país é:


,country,currency,average_cost_for_two
0,Indonesia,Indonesian Rupiah(IDR),303000.000000
1,Australia,Dollar($),138959.783333
2,Sri Lanka,Sri Lankan Rupee(LKR),2579.375000
3,Philippines,Botswana Pula(P),1227.825000
4,India,Indian Rupees(Rs.),704.400514
5,South Africa,Rand(R),339.228324
6,Qatar,Qatari Rial(QR),174.000000
7,United Arab Emirates,Emirati Diram(AED),153.716667
8,Singapure,Dollar($),141.437500
9,Brazil,Brazilian Real(R$),138.812500


## Cidades

### 1) Qual o nome da cidade que possui mais restaurantes registrados?

In [42]:
df_aux = (df2.loc[:, ['restaurant_id', 'city'] ].groupby('city')
                                                .count()
                                                .sort_values('restaurant_id', ascending=False)
                                                .reset_index())
print("A cidade que possui mais restaurantes registrados é {}.".format(df_aux.iloc[0,0]))

A cidade que possui mais restaurantes registrados é Abu Dhabi.


### 2) Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [43]:
df_aux = (df2.loc[df2['aggregate_rating'] >= 4, ['restaurant_id', 'city'] ].groupby('city')
                                                                 .count()
                                                                 .sort_values('restaurant_id', ascending=False)
                                                                 .reset_index())
print("A cidade que possui mais restaurantes com nota média acima de 4 é {}.".format(df_aux.iloc[0,0]))

A cidade que possui mais restaurantes com nota média acima de 4 é London.


### 3) Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [44]:
df_aux = (df2.loc[df2['aggregate_rating'] <= 2.5, ['restaurant_id', 'city'] ].groupby('city')
                                                                 .count()
                                                                 .sort_values('restaurant_id', ascending=False)
                                                                 .reset_index())
print("A cidade que possui mais restaurantes com nota média abaixo de 2.5 é {}.".format(df_aux.iloc[0,0]))

A cidade que possui mais restaurantes com nota média abaixo de 2.5 é Gangtok.


### 4) Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [45]:
df_aux = (df2.loc[:, ['average_cost_for_two', 'city'] ].groupby('city')
                                                       .mean()
                                                       .sort_values('average_cost_for_two', ascending=False)
                                                       .reset_index())

print("A cidade que possui o maior valor médio de um prato para dois é {}.".format(df_aux.iloc[0,0]))

A cidade que possui o maior valor médio de um prato para dois é Adelaide.


### 5) Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [46]:
df_aux = (df2.loc[:, ['cuisines', 'city'] ].groupby('city')
                                           .nunique()
                                           .sort_values('cuisines', ascending=False)
                                           .reset_index())
print("A cidade que possui a maior quantidade de tipos de culinária distintas é {}.".format(df_aux.iloc[0,0]))

A cidade que possui a maior quantidade de tipos de culinária distintas é Birmingham.


### 6) Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [47]:
df_aux = (df2.loc[df2['has_table_booking'] == 1, ['restaurant_id', 'city'] ].groupby('city')
                                                                     .count()
                                                                     .sort_values('restaurant_id', ascending=False)
                                                                     .reset_index())

print("A cidade que possui a maior quantidade de restaurantes que fazem reservas é {}.".format(df_aux.iloc[0,0]))

A cidade que possui a maior quantidade de restaurantes que fazem reservas é Bangalore.


### 7) Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [48]:
df_aux = (df2.loc[df2['is_delivering_now'] == 1, ['restaurant_id', 'city'] ].groupby('city')
                                                                 .count()
                                                                 .sort_values('restaurant_id', ascending=False)
                                                                 .reset_index())
print("A cidade que possui a maior quantidade de restaurantes que fazem entregas é {}.".format(df_aux.iloc[0,0]))

A cidade que possui a maior quantidade de restaurantes que fazem entregas é Vadodara.


### 8) Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [49]:
df_aux =(df2.loc[df2['has_online_delivery'] == 1, ['restaurant_id', 'city'] ].groupby('city')
                                                                     .count()
                                                                     .sort_values('restaurant_id', ascending=False)
                                                                     .reset_index())
print("A cidade que possui a maior quantidade de restaurantes que aceitam pedidos online é {}.".format(df_aux.iloc[0,0]))

A cidade que possui a maior quantidade de restaurantes que aceitam pedidos online é Bhopal.


## Restaurantes

### 1) Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [51]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'aggregate_rating', 'votes']

df_aux = df2.loc[:, cols].sort_values(['votes', 'restaurant_id'], ascending=[False, True])
print("O restaurante que possui a maior quantidade de avaliações é {}.".format(df_aux.iloc[0, 1]))

O restaurante que possui a maior quantidade de avaliações é Bawarchi.


### 2) Qual o nome do restaurante com a maior nota média?

In [52]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'aggregate_rating', 'votes']

df_aux = df2.loc[:, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("O restaurante com a maior nota média é {}.".format(df_aux.iloc[0, 1]))

O restaurante com a maior nota média é Indian Grill Room.


### 3) Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [53]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

df_aux = df2.loc[:, cols].sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True])
print("O restaurante que possui o maior valor de uma prato para duas pessoas é {}.".format(df_aux.iloc[0, 1]))

O restaurante que possui o maior valor de uma prato para duas pessoas é d'Arry's Verandah Restaurant.


### 4) Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [54]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

df_aux = df2.loc[:, cols].sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True])
print("O restaurante de tipo de culinária brasileira que possui a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

O restaurante de tipo de culinária brasileira que possui a menor média de avaliação é d'Arry's Verandah Restaurant.


### 5) Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [56]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['country'] == 'Brazil') & (df2['cuisines'] == 'Brazilian')

df_aux = df2.loc[lines, cols].sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True])
print("O restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

O restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação é Oro.


### 6) Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [57]:
df_aux = (df2.loc[:, ['votes', 'has_online_delivery'] ].groupby('has_online_delivery')
                                                       .mean()
                                                       .sort_values('votes', ascending=False)
                                                       .reset_index())
print('Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas:')
display(df_aux)
print('Como podemos observar no quadro acima realmente são eles.')

Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas:


,has_online_delivery,votes
0,1,838.821664
1,0,479.421018


Como podemos observar no quadro acima realmente são eles.


### 7) Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [58]:
df_aux = (df2.loc[:, ['average_cost_for_two', 'has_table_booking'] ].groupby('has_table_booking')
                                                                    .mean()
                                                                    .sort_values('average_cost_for_two', ascending=False)
                                                                    .reset_index())
print('Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas:')
display(df_aux)
print('Como podemos observar no quadro acima realmente são eles.')

Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas:


,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


Como podemos observar no quadro acima realmente são eles.


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [60]:
lines = (df2['country'] == 'United States of America') & ((df2['cuisines'] == 'BBQ') | (df2['cuisines'] == 'Japanese'))

df_aux = (df2.loc[lines, ['average_cost_for_two', 'cuisines'] ].groupby('cuisines')
                                                               .mean()
                                                               .sort_values('average_cost_for_two', ascending=False)
                                                               .reset_index())
print('Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ):')
display(df_aux)
print('Como podemos observar no quadro acima realmente possuem.')

Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ):


,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


Como podemos observar no quadro acima realmente possuem.


## Tipo de Colunária

### 1) Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [61]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Italian')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("Dos restaurantes que possuem o tipo de culinária italiana, o restaurante com a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária italiana, o restaurante com a maior média de avaliação é Darshan.


### 2) Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [62]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Italian')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])
print("Dos restaurantes que possuem o tipo de culinária italiana, o restaurante com a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária italiana, o restaurante com a menor média de avaliação é Avenida Paulista.


### 3) Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [64]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'American')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("Dos restaurantes que possuem o tipo de culinária americana, o restaurante com a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária americana, o restaurante com a maior média de avaliação é Burger & Lobster.


### 4) Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [65]:
cols = ['restaurant_id', 'restaurant_name', 'country','city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'American')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])
print("Dos restaurantes que possuem o tipo de culinária americana, o restaurante com a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária americana, o restaurante com a menor média de avaliação é Alston Bar & Beef.


### 5) Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [66]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two','aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Arabian')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("Dos restaurantes que possuem o tipo de culinária árabe, o restaurante com a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária árabe, o restaurante com a maior média de avaliação é Mandi@36.


### 6) Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [67]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Arabian')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])
print("Dos restaurantes que possuem o tipo de culinária árabe, o restaurante com a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária árabe, o restaurante com a menor média de avaliação é Raful.


### 7) Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [68]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Japanese')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("Dos restaurantes que possuem o tipo de culinária japonesa, o restaurante com a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária japonesa, o restaurante com a maior média de avaliação é Sushi Samba.


### 8) Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [70]:
cols = ['restaurant_id', 'restaurant_name', 'country','city', 'cuisines', 'average_cost_for_two','aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Japanese')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])
print("Dos restaurantes que possuem o tipo de culinária japonesa, o restaurante com a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária japonesa, o restaurante com a menor média de avaliação é Banzai Sushi.


### 9) Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [71]:
cols = ['restaurant_id', 'restaurant_name', 'country','city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Home-made')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
print("Dos restaurantes que possuem o tipo de culinária caseira, o restaurante com a maior média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária caseira, o restaurante com a maior média de avaliação é Kanaat Lokantası.


### 10) Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [72]:
cols = ['restaurant_id', 'restaurant_name', 'country', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']

lines = (df2['cuisines'] == 'Home-made')

df_aux = df2.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])
print("Dos restaurantes que possuem o tipo de culinária caseira, o restaurante com a menor média de avaliação é {}.".format(df_aux.iloc[0, 1]))

Dos restaurantes que possuem o tipo de culinária caseira, o restaurante com a menor média de avaliação é GurMekan Restaurant.


### 11) Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [75]:
df_aux = (df2.loc[:, ['average_cost_for_two', 'cuisines'] ].groupby('cuisines')
                                                  .max()
                                                  .sort_values('average_cost_for_two', ascending=False)
                                                  .reset_index())
print("O tipo de culinária que possui o maior valor médio de um prato para duas pessoas é {}.".format(df_aux.iloc[0, 0]))

O tipo de culinária que possui o maior valor médio de um prato para duas pessoas é Modern Australian.


### 12) Qual o tipo de culinária que possui a maior nota média?

In [76]:
df_aux = (df2.loc[:, ['aggregate_rating', 'cuisines'] ].groupby('cuisines')
                                                       .max()
                                                       .sort_values('aggregate_rating', ascending=False)
                                                       .reset_index())
print("O tipo de culinária que possui a maior nota média é {}.".format(df_aux.iloc[0, 0]))

O tipo de culinária que possui a maior nota média é Indian.


### 13) Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [77]:
lines = (df2['has_online_delivery'] == 1) & (df2['is_delivering_now'] == 1)

df_aux = (df2.loc[:, ['restaurant_id', 'cuisines'] ].groupby('cuisines')
                                                    .count()
                                                    .sort_values('restaurant_id', ascending=False)
                                                    .reset_index())
print("O tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas é {}.".format(df_aux.iloc[0, 0]))

O tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas é North Indian.
